<h1> Analyzing the Neighborhoods in Dubai for setting up a New Fast Food Restaurant </h1>
          <h2>IBM Applied Data Science Capstone Project</h2>
          <h3>By : Ahmad Kanbari </h3>

Since the first of january 2021, I have been working to achieve an IBM Data Science Professional certification. The course teaches what it means to be a data scientist, and the tools that you need to succeed in this field.The classes included Python programming, SQL databases, machine learning, statistics, and more.  As a Capstone for the certification, students are required to define a business problem and pull data from the web and location data from Foursquare to solve it. This post details my process for determining the best spot in Dubai to open a Fast Food Restaurant, from problem statement to data preparation to analysis to recommendations. Detailed code for this project is located on GitHub and can be found at the end of this post.

# <h1>1- Introduction</h1>

<h2>1.1- Business Problem</h2>

Food is a lucrative business industry and the Middle East is the hub of delicious culinary delights from all over the world.
Now adays with covid-19 presence,Fast-food restaurants has proven to provid safe and quick meals to people on the go and offer an alternative to traditional, sit-down restaurants. 
Imagine you have open a fast food restaurant  or a food truck whether its Western fast food style  where you serve burger and drinks or its a middle eastern style where you sell shawarma,falafel and other sandwiches.
With following the food code of UAE and choosing right place, This business will be so profitable  as it provides tasty food in a quick, low-cost manner.

Before opening a fast-food restaurant, however, research the city and determine if your fast-food business has enough potential demand from the population.So what factors should you analyze to determine the best place in the city to start your fast food restuarant, and how do you do it? This report outlines some basic assumptions, data sets, and analysis that can inform your decision when selecting the optimal neighborhood in Dubai for opening a Fast Food restaurant or your food truck.



<h1>1.2- Target Audience </h1>

The target audience of this report would be anyone who wants to buy or build a fast food restaurant or fast food truck in Dubai.


<h1>2- Data</h1>

The dataset that I used in this project is of Dubai communities scrapped from Wikipedia. This dataset contains the list of 131 communities of Dubai.

Data source:

https://en.wikipedia.org/wiki/List_of_communities_in_Dubai

I scrapped the data from the table of Wikipedia using a python library called ‘Beautiful soup’. We will use only 4 columns of the dataset i.e. Community Number, Community (English) and Community (Arabic) for venue analysis of Dubai and the population density column.
Following data sources will be needed to extract/generate the required information:

- I Will use GEOPY python library to generate Latitude and Longitude for the Abu Dhabi Island
- I Will use the GEOCODER python library to geocode coordinates (Latitudes and Longitude for schools in Abu Dhabi.
- number of venues and their type and location in every neighborhood will be obtained using FOURSQUARE API


#3- Methedology


In [ ]:
#import required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

#Import Beautiful Soup, lxml, requests to scrap data from Toronto Neighborhood in Wikipedia

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium

import folium # map rendering library

print("Libraries Imported!")

Libraries Imported!


#1- The first step in this project is to scrap the data from wikipedia page then clean it up by dropping unnecessary columns, renaming the remaining columns and getting rid of NaN values .




In [ ]:
url='https://en.wikipedia.org/wiki/List_of_communities_in_Dubai'
response=requests.get(url).text

soup=BeautifulSoup(response, 'html.parser')


In [ ]:
Dubaitable=soup.find('table',{'class':'wikitable sortable'})


In [ ]:
df_Dubai = pd.read_html('https://en.wikipedia.org/wiki/List_of_communities_in_Dubai')[0]


In [ ]:
df_Dubai.shape

(153, 7)

In [ ]:
df_Dubai.head()


,Community Number,Community (English),Community (Arabic),Area(km2),Population(2000),Population density(/km2),Unnamed: 6
0,126.0,Abu Hail,أبو هيل,1.27 km2,21414,"16,861.4/km2",NaN
1,711.0,Al Awir First,العوير الأولى,NaN,NaN,NaN,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN,NaN,NaN,NaN
3,283.0,Aleyas,العياص,162.4 km2,1706,162.4/km2,NaN
4,333.0,Al Bada'a,البدع,0.82 km2,18816,22946/km2,NaN


In [ ]:
#dropping the non-required columns
df_Dubai.drop(['Area(km2)','Population(2000)','Unnamed: 6'], axis=1, inplace=True)

In [ ]:
df_Dubai.head()


,Community Number,Community (English),Community (Arabic),Population density(/km2)
0,126.0,Abu Hail,أبو هيل,"16,861.4/km2"
1,711.0,Al Awir First,العوير الأولى,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN
3,283.0,Aleyas,العياص,162.4/km2
4,333.0,Al Bada'a,البدع,22946/km2


In [ ]:

#renaming the columns
df_Dubai.rename(columns={'Community Number':'Neighborhood_Number','Community (English)':'Neighborhoods','Community (Arabic)':'Neighborhood_Arabic','Population density(/km2)':'Population_density'}, inplace=True)
df_Dubai.head(131)

,Neighborhood_Number,Neighborhoods,Neighborhood_Arabic,Population_density
0,126.0,Abu Hail,أبو هيل,"16,861.4/km2"
1,711.0,Al Awir First,العوير الأولى,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN
3,283.0,Aleyas,العياص,162.4/km2
4,333.0,Al Bada'a,البدع,22946/km2
5,122.0,Al Baraha,البراحة,"7,086/km2"
6,373.0,Al Barsha First,البرشاء الأولى,33/km2
7,376.0,Al Barsha Second,البرشاء الثانية,33/km2
8,375.0,Al Barsha Third,البرشاء الثالثة,33/km2
9,671.0,Al Barsha South First,البرشاء جنوب الاولى,33/km2


In [ ]:
#Cleaning Data
df_Dubai.isnull().sum()


Neighborhood_Number     1
Neighborhoods           0
Neighborhood_Arabic    10
Population_density     48
dtype: int64

In [ ]:
df_Dubai['Neighborhood_Number'] = df_Dubai['Neighborhood_Number'].replace(np.nan, 1)


In [ ]:
df_Dubai.head(7)

,Neighborhood_Number,Neighborhoods,Neighborhood_Arabic,Population_density
0,126.0,Abu Hail,أبو هيل,"16,861.4/km2"
1,711.0,Al Awir First,العوير الأولى,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN
3,283.0,Aleyas,العياص,162.4/km2
4,333.0,Al Bada'a,البدع,22946/km2
5,122.0,Al Baraha,البراحة,"7,086/km2"
6,373.0,Al Barsha First,البرشاء الأولى,33/km2


In [ ]:
df_Dubai.shape

(153, 4)

#2- The second step is to get the longitude and latitude of dubi city and each nieghborhood in dubai.

In [ ]:
#defining the latitude and longitude of City/place whose map is to be created
address = 'Dubai'

geolocator = Nominatim(user_agent="dubai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai are 25.074282349999997, 55.18853865430702.


In [ ]:
#Get Latitude and Longitude for communitites and ignoring the communities whose data is not available
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderNotFound

address= df_Dubai['Neighborhoods']
geolocater= Nominatim(user_agent="dubai-explorer")
location=[]
empty=[]


def getcoords(add):
    try:
        coords= geolocater.geocode(add, timeout=10)
        location.append([add, coords.latitude, coords.longitude])
        print("the coordinates are {}".format(location[-1]))
        

    except GeocoderTimedOut:
        return getcoords(add)
    
    except:
        empty.append([add])
        print("Couldn't find coordinates of {}".format(empty[-1]))
        
for add in address:
        getcoords(add)

the coordinates are ['Abu Hail', 25.2860287, 55.328865]
the coordinates are ['Al Awir First', 25.185184200000002, 55.5651697615552]
Couldn't find coordinates of ['Al Awir Second']
the coordinates are ['Aleyas', 25.2117884, 55.536023378308464]
the coordinates are ["Al Bada'a", 25.2246261, 55.2684831]
the coordinates are ['Al Baraha', 25.2810618, 55.3194665]
the coordinates are ['Al Barsha First', 25.1117235, 55.2064271]
Couldn't find coordinates of ['Al Barsha Second']
Couldn't find coordinates of ['Al Barsha Third']
the coordinates are ['Al Barsha South First', 25.0747157, 55.2320479]
Couldn't find coordinates of ['Al Barsha South Second']
Couldn't find coordinates of ['Al Barsha South Third']
Couldn't find coordinates of ['Al Barsha South Fourth']
Couldn't find coordinates of ['Al Barsha South Fifth']
the coordinates are ['Al Buteen', 25.26305655, 55.3205840389995]
the coordinates are ['Al Corniche', 25.6821827, 51.5161526]
the coordinates are ['Al Dhagaya', 25.2726408, 55.3002405]
th

In [ ]:
Dubai_data= pd.DataFrame(location, columns=['Neighborhoods','Latitude','Longitude'])
Dubai_data.to_csv('Dubai_neighborhoods_locations.csv')


In [ ]:
Dubai_data.head(5)


,Neighborhoods,Latitude,Longitude
0,Abu Hail,25.286029,55.328865
1,Al Awir First,25.185184,55.565170
2,Aleyas,25.211788,55.536023
3,Al Bada'a,25.224626,55.268483
4,Al Baraha,25.281062,55.319466


#2.1- Then we can create a map showing each neighborhoods in Dubai


In [ ]:
# create map of Dubai latitude and longitude values
map_Dubai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Dubai_data['Latitude'], Dubai_data['Longitude'], Dubai_data['Neighborhoods']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_Dubai)
        

In [ ]:
map_Dubai

In [ ]:
Dubai_data.head()


,Neighborhoods,Latitude,Longitude
0,Abu Hail,25.286029,55.328865
1,Al Awir First,25.185184,55.565170
2,Aleyas,25.211788,55.536023
3,Al Bada'a,25.224626,55.268483
4,Al Baraha,25.281062,55.319466


In [ ]:
CLIENT_ID = 'R4UYUND5LBHL0YHL1NYYAIKQQCZEJDAXQ1WULAAMMZ4DBF3L' # your Foursquare ID
CLIENT_SECRET = 'JVQJYBLRH5N0YH02LOJCZRHD42NON13OU0KTRB2P543QRTA2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R4UYUND5LBHL0YHL1NYYAIKQQCZEJDAXQ1WULAAMMZ4DBF3L
CLIENT_SECRET:JVQJYBLRH5N0YH02LOJCZRHD42NON13OU0KTRB2P543QRTA2


#3- Doing some exploratory anaylsis for a neighborhood in dubai



In [ ]:
#exploring the venues of 'Al Awir First, Dubai'
Dubai_data.loc[1,'Neighborhoods']


'Al Awir First'

In [ ]:
neighborhood_latitude = Dubai_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Dubai_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Dubai_data.loc[1, 'Neighborhoods'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Al Awir First are 25.2860287, 55.328865.


In [ ]:
LIMIT=100 # limit of number of venues returned by Foursquare API
radius=500

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighborhood_latitude,
neighborhood_longitude,
radius,
LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=R4UYUND5LBHL0YHL1NYYAIKQQCZEJDAXQ1WULAAMMZ4DBF3L&client_secret=JVQJYBLRH5N0YH02LOJCZRHD42NON13OU0KTRB2P543QRTA2&v=20180605&ll=25.2860287,55.328865&radius=500&limit=100'

In [ ]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c3c66b5928183be006aeca'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5f74345de856803a49d07f83-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '52f2ab2ebcbc57f1066b8b46',
         'name': 'Supermarket',
         'pluralName': 'Supermarkets',
         'primary': True,
         'shortName': 'Supermarket'}],
       'id': '5f74345de856803a49d07f83',
       'location': {'address': 'Abu Hail',
        'cc': 'AE',
        'city': 'الخبيصي',
        'country': 'الإمارات العربية المتحدة',
        'distance': 71,
        'formattedAddress': ['Abu Hail',
         'الخبيصي',
         'الإمارات العربية المتحدة'],
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues=results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-5f74345de856803a49d07f83-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
      'suffix': '.png'},
     'id': '52f2ab2ebcbc57f1066b8b46',
     'name': 'Supermarket',
     'pluralName': 'Supermarkets',
     'primary': True,
     'shortName': 'Supermarket'}],
   'id': '5f74345de856803a49d07f83',
   'location': {'address': 'Abu Hail',
    'cc': 'AE',
    'city': 'الخبيصي',
    'country': 'الإمارات العربية المتحدة',
    'distance': 71,
    'formattedAddress': ['Abu Hail', 'الخبيصي', 'الإمارات العربية المتحدة'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 25.2858687,
      'lng': 55.3281742}],
    'lat': 25.2858687,
    'lng': 55.3281742,
    'state': 'دبي'},
   'name': 'Al Douri Mart Supermarket & Roastery',
   'photos': {'count': 0, 'groups': []},

In [ ]:
#Now we are ready to clean the json and structure it into a pandas dataframe.
nearby_venues=json_normalize(venues)
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Al Douri Mart Supermarket & Roastery,Supermarket,25.285869,55.328174
1,Baithak Restaurant,Asian Restaurant,25.288937,55.327372
2,Lively,Track,25.285194,55.325276
3,Jannati Health Club and Spa,Spa,25.285408,55.325168
4,Hamriya Park,Park,25.285710,55.333000


In [ ]:
#we can see we have got only four venues in Abu Hail within 500m radius
print("{} venues were returned by foursquare".format(nearby_venues.shape[0]))

6 venues were returned by foursquare


#4- The fourth step is to get all nearby venues of all neighborhoods of Dubai city in range 500 m  , then group these venues with by category

In [ ]:

print(Dubai_data.shape)

(101, 3)


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT1=10
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT1)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#get the near by venues of above data frame i.e. dubai_data1
Dubai_venues = getNearbyVenues(names=Dubai_data['Neighborhoods'],
                                   latitudes=Dubai_data['Latitude'],
                                   longitudes=Dubai_data['Longitude']
                                  )

Abu Hail
Al Awir First
Aleyas
Al Bada'a
Al Baraha
Al Barsha First
Al Barsha South First
Al Buteen
Al Corniche
Al Dhagaya
Al Fagaa
Al Garhoud
Al Hamriya, Dubai
Al Hamriya Port
Al Hathmah
Al Hudaiba
Al Jaddaf
Al Jafiliya
Al Karama
Al Khabisi
Al Khawaneej First
Al Kifaf
Al Mamzar
Al Manara
Al Mankhool
Al Merkad
Al Mina
Al Mizhar First
Al Muraqqabat
Al Murar
Al Mushrif
Al Muteena
Al Nahda First
Al Nahda Second
Al Nasr, Dubai
Al Quoz First
Al Quoz Industrial First
Al Qusais First
Al Qusais Industrial First
Al Qusais Industrial Third
Al Qusais Third
Al Raffa
Al Ras
Al Rashidiya
Al Rigga
Al Sabkha
Al Safa First
Al Safa Second
Al Satwa
Al Shindagha
Al Souq Al Kabeer
Al Twar First
Al Warqa'a First
Al Warqa'a Third
Al Wasl
Al Waheda
Ayal Nasir
Business Bay
Bu Kadra
Downtown Dubai
Dubai Investment Park First
Dubai Investment Park Second
Emirates Hill First
Emirates Hill Second
Emirates Hill Third
Hatta
Hor Al Anz
Hor Al Anz East
Jebel Ali 1
Jebel Ali 2
Jebel Ali Industrial
Jebel Ali Palm
Jumeira 

In [ ]:
print(Dubai_venues.shape)
Dubai_venues.head()

(557, 7)


,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abu Hail,25.286029,55.328865,Al Douri Mart Supermarket & Roastery,25.285869,55.328174,Supermarket
1,Abu Hail,25.286029,55.328865,Baithak Restaurant,25.288937,55.327372,Asian Restaurant
2,Abu Hail,25.286029,55.328865,Lively,25.285194,55.325276,Track
3,Abu Hail,25.286029,55.328865,Jannati Health Club and Spa,25.285408,55.325168,Spa
4,Abu Hail,25.286029,55.328865,Hamriya Park,25.285710,55.333000,Park


In [ ]:


#finiding the no. of various fastfood restaurants in Dubai
fastfood_restaurant = Dubai_venues[Dubai_venues['Venue Category'].str.contains('Fast Food Restaurant')].reset_index(drop=True)
fastfood_restaurant.index = np.arange(1, len(fastfood_restaurant )+1)

In [ ]:
print (fastfood_restaurant['Venue Category'].value_counts())


Fast Food Restaurant    19
Name: Venue Category, dtype: int64


<h1>Analysis and Clustering</h1>





#5- In the fifth step we will analyze the data , by digging further into each of the neighborhoods to see the most popular types of venues for each neighborhood. To do this, we will take the following steps:



*   Create a data frame of venue categories with pandas one hot encoding
*   Use pandas groupby to get the mean of the one-hot encoded venue categories
*   Transpose the data frame and arrange in descending order









In [ ]:
# one hot encoding
dubai_onehot = pd.get_dummies(Dubai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['Neighborhoods'] = Dubai_venues['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]

dubai_onehot.head()

,Neighborhoods,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Beach,Bookstore,Boutique,Bridal Shop,Burger Joint,Burrito Place,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cupcake Shop,Currency Exchange,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Donburi Restaurant,Donut Shop,Drugstore,Egyptian Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lingerie Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Museum,Nail Salon,Nightclub,North Indian Restaurant,Outdoors & Recreation,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Photography Lab,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Supermarket,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Abu Hail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Abu Hail,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abu Hail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Abu Hail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abu Hail,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dubai_onehot.shape

(557, 147)

In [ ]:
dubai_grouped = dubai_onehot.groupby('Neighborhoods').mean().reset_index()
dubai_grouped

,Neighborhoods,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Basketball Court,Beach,Bookstore,Boutique,Bridal Shop,Burger Joint,Burrito Place,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cupcake Shop,Currency Exchange,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Donburi Restaurant,Donut Shop,Drugstore,Egyptian Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lingerie Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Museum,Nail Salon,Nightclub,North Indian Restaurant,Outdoors & Recreation,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Photography Lab,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Supermarket,Syrian Restaurant,Tapas Restaurant,Thai Restaurant,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Abu Hail,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.166667,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,Al Awir First,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.250000,0.0,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,Al Bada'a,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.200000,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
dubai_grouped.shape

(81, 147)

In [ ]:
num_top_venues = 5
i =1
for hood in dubai_grouped['Neighborhoods']:
    print("----"+hood+"----",i)
    i = 1+i
    temp = dubai_grouped[dubai_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abu Hail---- 1
              venue  freq
0              Park  0.33
1               Spa  0.17
2  Asian Restaurant  0.17
3             Track  0.17
4       Supermarket  0.17


----Al Awir First---- 2
                  venue  freq
0     Convenience Store  0.25
1  Fast Food Restaurant  0.25
2    Seafood Restaurant  0.25
3           Coffee Shop  0.25
4  Pakistani Restaurant  0.00


----Al Bada'a---- 3
                       venue  freq
0                       Park   0.4
1                       Pool   0.2
2  Middle Eastern Restaurant   0.2
3                       Café   0.2
4      Outdoors & Recreation   0.0


----Al Baraha---- 4
                       venue  freq
0  Middle Eastern Restaurant   0.2
1                      Hotel   0.2
2                        Spa   0.1
3          Convenience Store   0.1
4                     Lounge   0.1


----Al Barsha First---- 5
            venue  freq
0  Ice Cream Shop   0.2
1       Juice Bar   0.1
2            Café   0.1
3   Burrito Place   0.1
4      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


                     venue  freq
0                     Park   1.0
1        Afghan Restaurant   0.0
2  North Indian Restaurant   0.0
3        Mobile Phone Shop   0.0
4      Moroccan Restaurant   0.0


----Al Wasl---- 47
                 venue  freq
0          Bridal Shop   0.2
1       Clothing Store   0.2
2                Hotel   0.2
3                 Food   0.2
4  Martial Arts School   0.2


----Arabian Ranches---- 48
                     venue  freq
0               Restaurant   0.4
1                     Pool   0.2
2              Golf Course   0.2
3               Sports Bar   0.2
4  North Indian Restaurant   0.0


----Ayal Nasir---- 49
               venue  freq
0  Afghan Restaurant   0.1
1               Café   0.1
2        Karaoke Bar   0.1
3         Hobby Shop   0.1
4          Nightclub   0.1


----Business Bay---- 50
                venue  freq
0               Hotel   0.3
1  Italian Restaurant   0.1
2  Chinese Restaurant   0.1
3         Coffee Shop   0.1
4              Bakery   0.1


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = dubai_grouped['Neighborhoods']

for ind in np.arange(dubai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck
1,Al Awir First,Coffee Shop,Fast Food Restaurant,Convenience Store,Seafood Restaurant,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
2,Al Bada'a,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
3,Al Baraha,Hotel,Middle Eastern Restaurant,Spa,Lounge,Convenience Store,Café,IT Services,American Restaurant,Arts & Crafts Store,Food Truck
4,Al Barsha First,Ice Cream Shop,Hotel,Juice Bar,Café,Burrito Place,Bakery,Egyptian Restaurant,Turkish Restaurant,Indian Restaurant,Fruit & Vegetable Store


<h1>Clustering The Neighborhood</h1>

## 6- In the sixth step we will use machine learning and cluster the the data into similar groups.

In [ ]:
#set number of clusters
kclusters = 5

dubai_grouped_clustering = dubai_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dubai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 4, 4, 4, 0, 2, 4, 2], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dubai_merged = Dubai_data

dubai_merged = dubai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

dubai_merged.head()# check the last columns!

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.286029,55.328865,1.0,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck
1,Al Awir First,25.185184,55.565170,2.0,Coffee Shop,Fast Food Restaurant,Convenience Store,Seafood Restaurant,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
2,Aleyas,25.211788,55.536023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Al Bada'a,25.224626,55.268483,1.0,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
4,Al Baraha,25.281062,55.319466,4.0,Hotel,Middle Eastern Restaurant,Spa,Lounge,Convenience Store,Café,IT Services,American Restaurant,Arts & Crafts Store,Food Truck


In [ ]:
#converting the 'Cluster Labels' column into int
dubai_merged['Cluster Labels'] = dubai_merged['Cluster Labels'].astype('Int64')

In [ ]:
#check for NaN rows and delete them
dubai_merged.isnull().sum(axis=1) # check the last columns!

0       0
1       0
2      11
3       0
4       0
5       0
6      11
7       0
8       0
9       0
10     11
11      0
12      0
13      0
14     11
15      0
16      0
17      0
18      0
19      0
20     11
21      0
22      0
23      0
24      0
25      0
26     11
27      0
28      0
29      0
30     11
31      0
32      0
33     11
34      0
35      0
36      0
37      0
38      0
39      0
40      0
41      0
42     11
43      0
44      0
45      0
46      0
47      0
48      0
49      0
50      0
51      0
52      0
53      0
54      0
55      0
56      0
57      0
58     11
59      0
60      0
61      0
62      0
63      0
64      0
65      0
66      0
67      0
68      0
69      0
70     11
71     11
72      0
73      0
74     11
75      0
76      0
77      0
78      0
79      0
80      0
81      0
82      0
83      0
84      0
85      0
86      0
87      0
88      0
89      0
90      0
91      0
92     11
93     11
94      0
95     11
96     11
97     11
98      0
99     11


In [ ]:
dubai_merged.dropna(inplace=True)
dubai_merged.reset_index(drop=True)
dubai_merged

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.286029,55.328865,1,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck
1,Al Awir First,25.185184,55.565170,2,Coffee Shop,Fast Food Restaurant,Convenience Store,Seafood Restaurant,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
3,Al Bada'a,25.224626,55.268483,1,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
4,Al Baraha,25.281062,55.319466,4,Hotel,Middle Eastern Restaurant,Spa,Lounge,Convenience Store,Café,IT Services,American Restaurant,Arts & Crafts Store,Food Truck
5,Al Barsha First,25.111724,55.206427,4,Ice Cream Shop,Hotel,Juice Bar,Café,Burrito Place,Bakery,Egyptian Restaurant,Turkish Restaurant,Indian Restaurant,Fruit & Vegetable Store
7,Al Buteen,25.263057,55.320584,4,Hotel,Ice Cream Shop,Restaurant,Asian Restaurant,Fried Chicken Joint,Coffee Shop,Middle Eastern Restaurant,Fish Market,Filipino Restaurant,Flea Market
8,Al Corniche,25.682183,51.516153,0,Cafeteria,Café,Fruit & Vegetable Store,Harbor / Marina,Beach,Fish Market,Asian Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
9,Al Dhagaya,25.272641,55.300241,2,Afghan Restaurant,Flea Market,Miscellaneous Shop,Shopping Mall,Flower Shop,Metro Station,Jewelry Store,Pakistani Restaurant,Indian Restaurant,Fast Food Restaurant
11,Al Garhoud,25.240737,55.352039,4,Dessert Shop,Hotel,Currency Exchange,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop
12,"Al Hamriya, Dubai",25.259930,55.303194,4,Persian Restaurant,Indian Restaurant,Pub,Beach,Coffee Shop,Juice Bar,Hotel,Middle Eastern Restaurant,Fast Food Restaurant,Electronics Store


In [ ]:
#let's visualize the cluster
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dubai_merged['Latitude'], dubai_merged['Longitude'], dubai_merged['Neighborhoods'], dubai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save('dubai_map.html')


<h1>Examine Clusters</h1> 

<h1>Cluster 1 </h1>

In [ ]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 0, dubai_merged.columns[[0] + list(range(1, dubai_merged.shape[1]))]]

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Al Corniche,25.682183,51.516153,0,Cafeteria,Café,Fruit & Vegetable Store,Harbor / Marina,Beach,Fish Market,Asian Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
27,Al Mizhar First,25.245084,55.450315,0,Café,Coffee Shop,Food,Pizza Place,Shopping Mall,Bakery,Salon / Barbershop,Burger Joint,French Restaurant,Food Truck
29,Al Murar,25.276739,55.310133,0,Café,Asian Restaurant,Hotel,Women's Store,Middle Eastern Restaurant,Pizza Place,Fast Food Restaurant,Filipino Restaurant,Electronics Store,Fish Market
43,Al Rashidiya,25.222025,55.391006,0,Nightclub,Basketball Court,Cafeteria,Café,Food,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Yoga Studio
52,Al Warqa'a First,25.252486,51.428843,0,Coffee Shop,Café,Burger Joint,Intersection,Sculpture Garden,Food Truck,Athletics & Sports,Yoga Studio,Frozen Yogurt Shop,Fried Chicken Joint
67,Hor Al Anz East,25.282082,55.347623,0,Middle Eastern Restaurant,Café,Performing Arts Venue,Burger Joint,Seafood Restaurant,Thai Restaurant,Smoke Shop,Yoga Studio,Food,Food Truck
72,Jumeira First,25.231409,55.262715,0,Salon / Barbershop,Café,Frozen Yogurt Shop,Burger Joint,Spa,Mobile Phone Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Filipino Restaurant,Fish Market
78,Nadd Al Shiba Third,24.469246,54.336866,0,Café,Coffee Shop,Performing Arts Venue,Middle Eastern Restaurant,Turkish Restaurant,Restaurant,Supermarket,Flea Market,Food Truck,Food & Drink Shop
88,Umm Hurair First,25.242212,55.321784,0,Pharmacy,Park,Café,Coffee Shop,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop
94,Za'abeel Second,24.660977,46.798415,0,Intersection,Café,Pakistani Restaurant,Yoga Studio,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop


<h1>Cluster 2 </h2>

In [ ]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 1, dubai_merged.columns[[0] + list(range(1, dubai_merged.shape[1]))]]

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.286029,55.328865,1,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck
3,Al Bada'a,25.224626,55.268483,1,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
46,Al Safa First,15.604586,32.557628,1,Garden,Soccer Stadium,Park,Concert Hall,BBQ Joint,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
47,Al Safa Second,29.324196,30.830316,1,Café,Park,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market
53,Al Warqa'a Third,25.190661,55.422755,1,Park,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market


<h1>Cluster 3 </h3>

In [ ]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 2, dubai_merged.columns[[0] + list(range(1, dubai_merged.shape[1]))]]

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Al Awir First,25.185184,55.565170,2,Coffee Shop,Fast Food Restaurant,Convenience Store,Seafood Restaurant,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
9,Al Dhagaya,25.272641,55.300241,2,Afghan Restaurant,Flea Market,Miscellaneous Shop,Shopping Mall,Flower Shop,Metro Station,Jewelry Store,Pakistani Restaurant,Indian Restaurant,Fast Food Restaurant
13,Al Hamriya Port,25.299532,55.334959,2,Arcade,Stables,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
15,Al Hudaiba,25.242833,55.279212,2,Korean Restaurant,Hotel,Smoke Shop,Middle Eastern Restaurant,Café,Lounge,Nightclub,Asian Restaurant,South American Restaurant,Grocery Store
17,Al Jafiliya,25.233594,55.292184,2,Coffee Shop,Fast Food Restaurant,Café,Sandwich Place,Track,Metro Station,Filipino Restaurant,Electronics Store,Fish Market,Flea Market
19,Al Khabisi,25.268333,55.336890,2,Sporting Goods Shop,Women's Store,Electronics Store,Fast Food Restaurant,Pool Hall,Currency Exchange,Café,Furniture / Home Store,Bookstore,Harbor / Marina
21,Al Kifaf,25.227887,55.274809,2,Electronics Store,Grocery Store,Indian Restaurant,Asian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Korean Restaurant,Yoga Studio,Food
23,Al Manara,33.195939,35.544608,2,Hostel,Bar,Trail,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop
25,Al Merkad,25.169213,55.292473,2,Metro Station,Baby Store,Food,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Flower Shop
28,Al Muraqqabat,25.265539,55.322513,2,Ice Cream Shop,Supermarket,Asian Restaurant,Fried Chicken Joint,Coffee Shop,Syrian Restaurant,Middle Eastern Restaurant,Iraqi Restaurant,Department Store,Fast Food Restaurant


<h1>Cluster 4 </h1>

In [ ]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 3, dubai_merged.columns[[0] + list(range(1, dubai_merged.shape[1]))]]

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Ras Al Khor,25.202378,55.359249,3,Shopping Plaza,Yoga Studio,Fruit & Vegetable Store,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop,Flea Market


<h1>Cluster 5 </h1>

In [ ]:
dubai_merged.loc[dubai_merged['Cluster Labels'] == 4, dubai_merged.columns[[0] + list(range(1, dubai_merged.shape[1]))]]

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Al Baraha,25.281062,55.319466,4,Hotel,Middle Eastern Restaurant,Spa,Lounge,Convenience Store,Café,IT Services,American Restaurant,Arts & Crafts Store,Food Truck
5,Al Barsha First,25.111724,55.206427,4,Ice Cream Shop,Hotel,Juice Bar,Café,Burrito Place,Bakery,Egyptian Restaurant,Turkish Restaurant,Indian Restaurant,Fruit & Vegetable Store
7,Al Buteen,25.263057,55.320584,4,Hotel,Ice Cream Shop,Restaurant,Asian Restaurant,Fried Chicken Joint,Coffee Shop,Middle Eastern Restaurant,Fish Market,Filipino Restaurant,Flea Market
11,Al Garhoud,25.240737,55.352039,4,Dessert Shop,Hotel,Currency Exchange,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop
12,"Al Hamriya, Dubai",25.259930,55.303194,4,Persian Restaurant,Indian Restaurant,Pub,Beach,Coffee Shop,Juice Bar,Hotel,Middle Eastern Restaurant,Fast Food Restaurant,Electronics Store
16,Al Jaddaf,25.221870,55.335925,4,Hotel,Harbor / Marina,Coffee Shop,Ice Cream Shop,Metro Station,Pub,French Restaurant,Food Truck,Food & Drink Shop,Food
18,Al Karama,25.248230,55.305157,4,Indian Restaurant,Ice Cream Shop,Dim Sum Restaurant,Chinese Restaurant,Asian Restaurant,Middle Eastern Restaurant,Yoga Studio,Food & Drink Shop,Frozen Yogurt Shop,Fried Chicken Joint
22,Al Mamzar,25.304353,55.364158,4,Dessert Shop,South Indian Restaurant,Burger Joint,Gym Pool,Hotel,Egyptian Restaurant,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market
24,Al Mankhool,25.249509,55.295477,4,Hotel,Burger Joint,Chinese Restaurant,Coffee Shop,Sports Bar,Supermarket,Hookah Bar,Indian Restaurant,Ice Cream Shop,Electronics Store
31,Al Muteena,25.274154,55.322556,4,Gym,African Restaurant,Hotel,Indian Restaurant,Fried Chicken Joint,Iraqi Restaurant,Hookah Bar,Electronics Store,Chinese Restaurant,Filipino Restaurant


# 7- After clustering we will select neighborhoods where parks is the one of the most first 3 venue category and try to find the ideal neighborhood for our business .



<h2>Let's find out the best placed for opening a fast food restaurant in Dubai</h2?

The best Neighborhoof to open a fast food restaurant will be a place where many people can gather froml diffirent places in dubai, so nieghborhoods where parks and shopping centers are located will be better for a fast food restaurant or food truck for being able to reach the maximum numbers of customers. also within those neighborhoods the ideal neighborhood will has the highest population Density.

In [105]:
#we will focus on neighborhoods that contain parks as the first and second most commmon venues 
#let's find out thoose neighborhoods with such criteria
dubai_most_common=neighborhoods_venues_sorted.iloc[:,:5]


In [106]:
dubai_most_common.head()

,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,1,Abu Hail,Park,Track,Spa
1,2,Al Awir First,Coffee Shop,Fast Food Restaurant,Convenience Store
2,1,Al Bada'a,Park,Middle Eastern Restaurant,Pool
3,4,Al Baraha,Hotel,Middle Eastern Restaurant,Spa
4,4,Al Barsha First,Ice Cream Shop,Hotel,Juice Bar


In [107]:
#1sr most common venue
df_group=dubai_most_common.groupby('1st Most Common Venue').count().reset_index()
#Arranging in ascending order
most_common1 = dubai_most_common.sort_values(['1st Most Common Venue'], ascending=[True])

In [108]:
#df_group
print(most_common1.shape)
most_common1

(81, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
48,2,Ayal Nasir,Afghan Restaurant,Hobby Shop,Indian Restaurant
7,2,Al Dhagaya,Afghan Restaurant,Flea Market,Miscellaneous Shop
9,2,Al Hamriya Port,Arcade,Stables,Yoga Studio
67,2,Naif,Asian Restaurant,Resort,Indian Restaurant
53,2,Emirates Hill First,Auto Garage,Warehouse Store,Currency Exchange
77,2,Umm Ramool,Auto Garage,Auto Workshop,Yoga Studio
68,2,Palm Jumeira,Beach,Outdoors & Recreation,Yoga Studio
28,2,Al Quoz Industrial First,Burger Joint,Gym,African Restaurant
6,0,Al Corniche,Cafeteria,Café,Fruit & Vegetable Store
72,0,Saih Al Salam,Café,Seafood Restaurant,Yoga Studio


In [109]:
#2nd most common venue
df_group2=dubai_most_common.groupby('2nd Most Common Venue').count().reset_index()
#Arranging in ascending order
most_common2 = dubai_most_common.sort_values(['2nd Most Common Venue'], ascending=[True])

In [110]:
#df_group
print(most_common2.shape)
most_common2

(81, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
24,4,Al Muteena,Gym,African Restaurant,Hotel
43,2,Al Waheda,Department Store,Art Museum,Sandwich Place
23,0,Al Murar,Café,Asian Restaurant,Hotel
77,2,Umm Ramool,Auto Garage,Auto Workshop,Yoga Studio
20,2,Al Merkad,Metro Station,Baby Store,Food
18,2,Al Manara,Hostel,Bar,Trail
34,0,Al Rashidiya,Nightclub,Basketball Court,Cafeteria
19,4,Al Mankhool,Hotel,Burger Joint,Chinese Restaurant
39,2,Al Satwa,Coffee Shop,Cafeteria,Furniture / Home Store
61,0,Jumeira First,Salon / Barbershop,Café,Frozen Yogurt Shop


In [111]:
#3rd most common venue
df_group3=dubai_most_common.groupby('3rd Most Common Venue').count().reset_index()
#Arranging in ascending order
most_common3 = dubai_most_common.sort_values(['3rd Most Common Venue'], ascending=[True])

In [112]:
#df_group
print(most_common3.shape)
most_common3

(81, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
28,2,Al Quoz Industrial First,Burger Joint,Gym,African Restaurant
22,2,Al Muraqqabat,Ice Cream Shop,Supermarket,Asian Restaurant
26,2,"Al Nasr, Dubai",Department Store,Pharmacy,Auto Garage
44,0,Al Warqa'a First,Coffee Shop,Café,Burger Joint
17,4,Al Mamzar,Dessert Shop,South Indian Restaurant,Burger Joint
34,0,Al Rashidiya,Nightclub,Basketball Court,Cafeteria
78,2,Umm Suqeim First,Middle Eastern Restaurant,Restaurant,Café
76,0,Umm Hurair First,Pharmacy,Park,Café
13,2,Al Jafiliya,Coffee Shop,Fast Food Restaurant,Café
49,4,Business Bay,Hotel,Café,Chinese Restaurant


In [113]:
#Neighborhood in Dubai famous for parks as 1st most common venue
df_parks1 = most_common1[most_common1['1st Most Common Venue'].str.contains('Park')].reset_index(drop=True)
print(df_parks1.shape)
df_parks1

(3, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,1,Al Warqa'a Third,Park,Yoga Studio,Flower Shop
1,1,Al Bada'a,Park,Middle Eastern Restaurant,Pool
2,1,Abu Hail,Park,Track,Spa


In [114]:
#Neighborhhod in Dubai famoud for parks as 2nd most common venue
df_parks2 = most_common1[most_common1['2nd Most Common Venue'].str.contains('Park')].reset_index(drop=True)
print(df_parks2.shape)
df_parks2

(3, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,1,Al Safa Second,Café,Park,Yoga Studio
1,2,Mirdif,Coffee Shop,Park,Seafood Restaurant
2,0,Umm Hurair First,Pharmacy,Park,Café


In [115]:
#Neighborhhod in Dubai famoud for parks as 3rd most common venue
df_parks3 = most_common1[most_common1['3rd Most Common Venue'].str.contains('Park')].reset_index(drop=True)
print(df_parks3.shape)
df_parks3

(1, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,1,Al Safa First,Garden,Soccer Stadium,Park


In [116]:
#let's concat data of all neighborhoods that contain parks as 1st 2nd or 3rd most common venue
#concatenating all 3 above tables 
dubai_parks_combined = pd.concat([df_parks1, df_parks2, df_parks3], ignore_index=True, sort=False)

In [117]:
print('The shape of dubai_parks_combined is: ', dubai_parks_combined.shape)
dubai_parks_combined.head(9)

The shape of dubai_parks_combined is:  (7, 5)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,1,Al Warqa'a Third,Park,Yoga Studio,Flower Shop
1,1,Al Bada'a,Park,Middle Eastern Restaurant,Pool
2,1,Abu Hail,Park,Track,Spa
3,1,Al Safa Second,Café,Park,Yoga Studio
4,2,Mirdif,Coffee Shop,Park,Seafood Restaurant
5,0,Umm Hurair First,Pharmacy,Park,Café
6,1,Al Safa First,Garden,Soccer Stadium,Park


In [118]:
#there are 7 neighborhood will a good optimal place to open a neighborhood 

#select full all data for those neighborhoods
df_neighborhoods_withparks = dubai_parks_combined['Neighborhoods']

df_neighborhoods_withparks




0    Al Warqa'a Third
1           Al Bada'a
2            Abu Hail
3      Al Safa Second
4              Mirdif
5    Umm Hurair First
6       Al Safa First
Name: Neighborhoods, dtype: object

In [119]:
hoods_with_parks_full =  dubai_merged[dubai_merged['Neighborhoods'].isin(df_neighborhoods_withparks)] 
hoods_with_parks_full.head(7)

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,25.286029,55.328865,1,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck
3,Al Bada'a,25.224626,55.268483,1,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food
46,Al Safa First,15.604586,32.557628,1,Garden,Soccer Stadium,Park,Concert Hall,BBQ Joint,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
47,Al Safa Second,29.324196,30.830316,1,Café,Park,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market
53,Al Warqa'a Third,25.190661,55.422755,1,Park,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market
76,Mirdif,25.221171,55.422420,2,Coffee Shop,Park,Seafood Restaurant,Café,Spa,Nail Salon,Gym,Grocery Store,Yoga Studio,Flower Shop
88,Umm Hurair First,25.242212,55.321784,0,Pharmacy,Park,Café,Coffee Shop,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop


In [120]:
#Add population density column to dataframe by joining it with df_duba dataframe
hoods_parks_areadensity = pd.merge(hoods_with_parks_full, df_Dubai, on='Neighborhoods', how='inner')
hoods_parks_areadensity.shape
hoods_parks_areadensity.head(7)

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood_Number,Neighborhood_Arabic,Population_density
0,Abu Hail,25.286029,55.328865,1,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,126.0,أبو هيل,"16,861.4/km2"
1,Al Bada'a,25.224626,55.268483,1,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,333.0,البدع,22946/km2
2,Al Safa First,15.604586,32.557628,1,Garden,Soccer Stadium,Park,Concert Hall,BBQ Joint,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,353.0,الصفا الأولى,"1,398/km2"
3,Al Safa Second,29.324196,30.830316,1,Café,Park,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market,357.0,الصفا الثانية,"1,398/km2"
4,Al Warqa'a Third,25.190661,55.422755,1,Park,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,Flea Market,423.0,الورقاء الثالثة,0.23/km2
5,Mirdif,25.221171,55.422420,2,Coffee Shop,Park,Seafood Restaurant,Café,Spa,Nail Salon,Gym,Grocery Store,Yoga Studio,Flower Shop,251.0,مردف,882/km2
6,Umm Hurair First,25.242212,55.321784,0,Pharmacy,Park,Café,Coffee Shop,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,314.0,أم هرير الأولى,1926/km2


#Discussion Section
Problem Which Tried to Solve:
The major purpose of this project, is to suggest the ideal neighborhood in a dubai for who wants to open a fast food reataurant or food truck. being near popular venues where people get togethers , checking for population density and competitiveness are among important factors to choose the ideal neighborhood.

#Solution :

#option 1 : Choosing the ideal place based on less competitiveness
for this option we are going to use cluster 0 to choose the ideal neighborhood where food trucks and fast food restauarants are not among the 10 most common venues category and as table provided the Mirdif neighborhood is the only place where there are no food trucks or fast food restaurants as the ideal neighborhood.

Selecting this neighborhood may lead to rapid growth of your business due to lack of competition and due to be near a park.

In [121]:
ideal_place = hoods_parks_areadensity[hoods_parks_areadensity['Neighborhoods']== 'Mirdif']
ideal_place

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood_Number,Neighborhood_Arabic,Population_density
5,Mirdif,25.221171,55.42242,2,Coffee Shop,Park,Seafood Restaurant,Café,Spa,Nail Salon,Gym,Grocery Store,Yoga Studio,Flower Shop,251.0,مردف,882/km2


In [122]:
#let's examine our map for that neighborhood 

map_ideal = folium.Map(location=[latitude, longitude], zoom_start=11)



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ideal_place['Latitude'], ideal_place['Longitude'], ideal_place['Neighborhoods'], ideal_place['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_ideal)
       
map_ideal

#option2: choosing ideal place only based on population desnity
let's  sort our nighborhoods  on population denstiy (population per area) in descending order 


In [130]:
#let's sort our values depends on population density
hoods_parks_areadensity['Population_density'] = hoods_parks_areadensity['Population_density'].replace('/km2','').astype(float)
#hoods_parks_areadensity['Population_density'] = hoods_parks_areadensity['Population_density'].str.replace(',','').astype(float)



top_neighborhoodss = hoods_parks_areadensity.set_index('Population_density')

top_neighborhoodss = top_neighborhoodss.sort_values(
      by="Population_density",
      ascending=False )
#all our choices

In [132]:
#let's choose top 3 neighborhoods with top populatin density
top3_neighborhoods= top_neighborhoodss.iloc[:3,:]
top3_neighborhoods

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood_Number,Neighborhood_Arabic
Population_density,,,,,,,,,,,,,,,,
22946.0,Al Bada'a,25.224626,55.268483,1,Park,Middle Eastern Restaurant,Pool,Café,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,Food,333.0,البدع
16861.4,Abu Hail,25.286029,55.328865,1,Park,Track,Spa,Asian Restaurant,Supermarket,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,126.0,أبو هيل
1926.0,Umm Hurair First,25.242212,55.321784,0,Pharmacy,Park,Café,Coffee Shop,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop,314.0,أم هرير الأولى


In [136]:
#let's examine the map of all 3 differnet choices
map_top = folium.Map(location=[latitude, longitude], zoom_start=11)



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(top3_neighborhoods['Latitude'], top3_neighborhoods['Longitude'], top3_neighborhoods['Neighborhoods'], top3_neighborhoods['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_top)
       
map_top

#Results 
If you want to choose the ideal neighborhood where the competiton is low you should go with Mirdif neighborhood.
If you care more about population density you can choose among top neighborhoods where populatin desnsity is high such as Al Bada'a , Abu Hail or Umm Hurair First.

#Conclusion
Foursquare data and K-means algorithm, alongside with folium mapping tools can be powerful tools helping people decide best places for doing such a business.where Data is the drive of the decision. This approach can be replicated for various other tasks, e.g. comparing two different cities or choosing best neighborhood to live on.

# Finish

